In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import _extract_game_id
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]
regrown_game_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples.pddl'))
regrown_game_1024_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples-1024.pddl'))
print(len(real_game_texts), len(regrown_game_texts), len(regrown_game_texts) / 98, len(regrown_game_1024_texts), len(regrown_game_1024_texts) / 98)


98 6272 64.0 100352 1024.0


In [4]:
fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl']


,Index,src_file,game_name,domain_name,original_game_name,real,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,...,ast_ngram_constraints_n_6_score,ast_ngram_constraints_n_7_score,ast_ngram_terminal_n_2_score,ast_ngram_terminal_n_3_score,ast_ngram_terminal_n_4_score,ast_ngram_terminal_n_5_score,ast_ngram_scoring_n_2_score,ast_ngram_scoring_n_3_score,ast_ngram_scoring_n_4_score,ast_ngram_scoring_n_5_score
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,6172feb1665491d1efbce164-0,1,1,1,1,1.00,...,0.976714,0.984956,0.942266,0.970180,0.973574,0.978002,0.725160,0.761936,0.942521,0.975955
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,5f77754ba932fb2c4ba181d8-2,1,1,1,1,1.00,...,0.986443,0.986721,0.935275,0.965656,0.967074,0.978883,0.670913,0.801850,0.870098,0.924083
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,614b603d4da88384282967a7-3,1,1,1,1,0.00,...,0.976849,0.981247,0.000000,0.000000,0.000000,0.000000,0.452633,0.593800,0.814709,0.883131
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,5bc79f652885710001a0e82a-5,1,1,1,1,0.00,...,0.986600,0.986476,0.000000,0.000000,0.000000,0.000000,0.725160,0.761936,0.942521,0.975955
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,614dec67f6eb129c3a77defd-6,1,1,1,1,0.25,...,0.985217,0.983768,0.000000,0.000000,0.000000,0.000000,0.937248,0.947704,0.933456,0.976437


# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [5]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [6]:
FITNESS_MODEL_DATE_ID = '2023_03_10_2'
FITNESS_FEATURIZER_PATH = '../models/fitness_featurizer_2023_03_10.pkl.gz'


mcmc = MCMCRegrowthSampler(DEFAULT_ARGS,
    FITNESS_MODEL_DATE_ID, FITNESS_FEATURIZER_PATH, greedy_acceptance=True, 
    plateau_patience_steps=100, max_steps=10000,
    fitness_function_relative_path='..')  #   acceptance_temperature=10.0, 

In [7]:
n_samples = 10
mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Plateaued at step 343 with energy -24.60292
Plateaued at step 291 with energy -25.03177
Plateaued at step 199 with energy -25.48553
Plateaued at step 139 with energy -21.86280
Plateaued at step 385 with energy -20.68362
Plateaued at step 546 with energy -25.49575
Plateaued at step 222 with energy -22.02627
Plateaued at step 580 with energy -23.30713
Plateaued at step 461 with energy -24.15441
Plateaued at step 130 with energy -19.70413


In [26]:
mcmc.visualize_sample(2)

### Energy of visualized game: -25.486

### Top features contributing to the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                            | Features decreasing energy (= more real)                                         |
|---------------------------------------------------------------------|----------------------------------------------------------------------------------|
| pref_forall_used_incorrect: **0.196** = (1.000) * 0.196             | ast_ngram_full_n_7_score: **-13.067** = (1.000) * -13.067                        |
| predicate_under_modal_once_in_motion: **0.029** = (1.000) * 0.029   | ast_ngram_constraints_n_7_score: **-6.535** = (1.000) * -6.535                   |
| no_adjacent_same_modal: **0.025** = (1.000) * 0.025                 | all_variables_defined: **-1.833** = (1.000) * -1.833                             |
| predicate_under_modal_once_agent_holds: **0.015** = (1.000) * 0.015 | all_variables_used: **-1.523** = (1.000) * -1.523                                |
|                                                                     | no_two_number_operations: **-1.037** = (1.000) * -1.037                          |
|                                                                     | all_preferences_used: **-0.438** = (1.000) * -0.438                              |
|                                                                     | max_number_variables_types_quantified_variables_1: **-0.355** = (1.000) * -0.355 |
|                                                                     | ast_ngram_setup_n_5_score: **-0.284** = (1.000) * -0.284                         |
|                                                                     | max_depth_constraints_1: **-0.233** = (1.000) * -0.233                           |
|                                                                     | in_motion_arg_types_balls_constraints: **-0.201** = (1.000) * -0.201             |
|                                                                     | in_motion_arg_types_receptacles_constraints: **-0.155** = (1.000) * -0.155       |
|                                                                     | predicate_under_modal_hold_in_motion: **-0.042** = (1.000) * -0.042              |
|                                                                     | max_number_variables_types_quantified_types_1: **-0.037** = (1.000) * -0.037     |
|                                                                     | agent_holds_arg_types_balls_constraints: **-0.021** = (1.000) * -0.021           |

### Game:

```pddl
(define (game mcmc-2) (:domain medium-objects-room-v1)
(:setup
  (game-conserved
    (agent_holds ?v0)
  )
)
(:constraints
  (and
    (preference preference0
      (exists (?v1 - dodgeball ?v2 - doggie_bed)
        (then
          (hold (not (in_motion ?v2) ) )
          (once (and (agent_holds ?v1) (in_motion ?v1) ) )
          (hold (in_motion agent) )
        )
      )
    )
  )
)
(:terminal
  (>= (count preference0:dodgeball) 4 )
)
(:scoring
  (+ (* 15 (* 0 (count preference0:dodgeball) )
    )
    (count preference0:pink_dodgeball)
  )
)
)
```

In [16]:
mcmc.samples[0][0][3]

('(:setup',
 {'setup': {'statement': {'conserved_pred': {'pred': {'and_args': [{'pred': {'pred': {'arg_1': {'term': '?v0', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_term', 'pos': 3924, 'endpos': 3924, 'line': 3924, 'endline': 3924, 'alerts': []}}, 'arg_2': {'term': '?v1', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_term', 'pos': 3925, 'endpos': 3925, 'line': 3925, 'endline': 3925, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'predicate_on', 'pos': 3926, 'endpos': 3926, 'line': 3926, 'endline': 3926, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'predicate', 'pos': 3927, 'endpos': 3927, 'line': 3927, 'endline': 3927, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'super_predicate', 'pos': 3928, 'endpos': 3928, 'line': 3928, 'endline': 3928, 'alerts': []}}, {'pred': {'pred': {'arg_1': {'term': '?v2', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_term', 'pos': 3929, 'endpos': 3929, 'line': 3929, '

In [24]:
mcmc.fitness_featurizer.parse(mcmc.samples[0][0], '', True, False)

1 4
{('?v2', 'variable_type_def', 1662)} {('?v2', 'variable_type_def', 1662)} False


{'src_file': '',
 'game_name': 'mcmc-0',
 'domain_name': 'medium-objects-room-v1',
 'real': False,
 'all_variables_defined': 0,
 'all_variables_used': 1,
 'all_preferences_used': 1,
 'setup_objects_used': 0,
 'adjacent_once_found': 0,
 'no_adjacent_same_modal': 0,
 'starts_and_ends_once': 0,
 'length_of_then_modals_1': 0,
 'length_of_then_modals_2': 0,
 'length_of_then_modals_3': 0,
 'length_of_then_modals_4': 0,
 'length_of_then_modals_5': 0,
 'length_of_then_modals_6': 0,
 'length_of_then_modals_7': 0,
 'max_quantification_count_setup_0': 1,
 'max_quantification_count_setup_1': 0,
 'max_quantification_count_setup_2': 0,
 'max_quantification_count_setup_3': 0,
 'max_quantification_count_setup_4': 0,
 'max_quantification_count_setup_5': 0,
 'max_quantification_count_constraints_0': 0,
 'max_quantification_count_constraints_1': 1,
 'max_quantification_count_constraints_2': 0,
 'max_quantification_count_constraints_3': 0,
 'max_quantification_count_constraints_4': 0,
 'max_quantification

## Regrwoth from existing games

In [ ]:
n_samples = 5
start = 2
end = start + n_samples


for original_index in range(start, end):
    print(f'Generating samples starting from original index {original_index} (id {_extract_game_id(real_game_texts[original_index])})')
    mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True, initial_proposal=game_asts[original_index])

    # print()
    # for i in range(n_samples * (original_index - start), n_samples * (original_index - start + 1)):
    #     print(i, mcmc.samples[i][2])

In [ ]:
mcmc.visualize_sample(27)

In [ ]:
mcmc.feature_names[-5:]

In [ ]:
from argparse import Namespace
from src.fitness_features import *
args = Namespace(no_binarize=False, no_merge=False)
featurizer = build_fitness_featurizer(args)

In [ ]:
a = grammar_parser.parse(s)

In [ ]:
ps = {k:v for k, v in featurizer.parse(a, 'mcmc', True).items() if 'predicate_under_modal' in k}
len(ps)

In [ ]:
{k:v for k, v in ps.items() if v > 0}

In [ ]:
mcmc.samples[5][0][3][1].setup.exists_args.setup.and_args[0].setup.exists_vars

In [ ]:
mcmc._proposal_to_features(mcmc.samples[0][0])['ast_ngram_n_7_score']

In [ ]:
mcmc.fitness_featurizer.full_ast_registry[0].n_gram_model.score(mcmc.samples[0][0], log=True)

In [ ]:
mcmc.fitness_featurizer.preprocessors[0].scale_series_min_max_values

In [ ]:
# %prun -r 
n_samples = 5
start = 0
end = start + n_samples

for original_index in range(start, end):
    print(f'Generating samples starting from original index {original_index} (id {_extract_game_id(real_game_texts[original_index])})')
    mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True, initial_proposal=game_asts[original_index])

    print()
    for i in range(n_samples * (original_index - start), n_samples * (original_index - start + 1)):
        print(i, mcmc.samples[i][2])

In [ ]:
mcmc.visualize_sample(14)

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][0], ' '))

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][3], ' '))

In [ ]:
ngram_model = mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1]

sample_ngrams = [('constraints', 'and', 'forall', 'wall', 'and'), ('and', 'forall', 'wall', 'and', 'preference'), ('forall', 'wall', 'and', 'preference', 'preferenceName'), ('wall', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]

print(f'Sample ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in sample_ngrams])} | ')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print(f'Original ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in original_ngrams])}')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])
print()

for s in sample_ngrams[:]:
    if s in original_ngrams:
        sample_ngrams.remove(s)
        original_ngrams.remove(s)

print('Sample remaining ngrams')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print('Original remaining ngrams')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

In [ ]:
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
print('Original ngrams')
for s in original_ngrams:
    print(s, mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_counts[s], mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_logprobs[s])

In [ ]:
mcmc.visualize_sample(7)

In [ ]:
mcmc.visualize_sample(15)

### Notes from looking at these games
* **TODO:** debug the 'section-without' logic, it seems to not work
* **TODO:** add features for preferences with different lengths of modals

In [ ]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [ ]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [ ]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

In [ ]:
print([x[2] for x in mcmc_with_crossover.samples])